In [ ]:
import pandas as pd
import numpy as np
import re

excel1_path = r"Excel Containing CAD Vertices(Outer).xlsx"   # <-- So this to your actual excel CAD(Outer) path
xyz_points = pd.read_excel(excel1_path)

notepad1_path = r"Excel Containing CAD(Outer) stl file.stl"  # <-- So this to your actual CAD(Outer) stl file path
with open(notepad1_path, 'r') as file:
    stl_content = file.read()

normals = []
triangles = []

facet_blocks = stl_content.strip().split('facet normal')
for block in facet_blocks[1:]:
    lines = block.strip().split('\n')
    normal = list(map(float, lines[0].strip().split()))
    vertices = []
    for line in lines:
        if line.strip().startswith('vertex'):
            vertex = list(map(float, line.strip().split()[1:]))
            vertices.append(vertex)
    if len(vertices) == 3:
        normals.append(normal)
        triangles.append(vertices)

normals = np.array(normals)
triangles = np.array(triangles)


In [ ]:
triangle_centroids = np.mean(triangles, axis=1)

results = []

for idx, point in xyz_points.iterrows():
    point_coords = np.array([point['X'], point['Y'], point['Z']])
    
    distances = np.linalg.norm(triangle_centroids - point_coords, axis=1)
    
    closest_idx = np.argmin(distances)
    
    closest_normal = normals[closest_idx]
    
    results.append({
        'X': point['X'],
        'Y': point['Y'],
        'Z': point['Z'],
        'Normal_X': closest_normal[0],
        'Normal_Y': closest_normal[1],
        'Normal_Z': closest_normal[2]
    })

output_df = pd.DataFrame(results)

In [ ]:
excel3_path =r"Excel Storing X,Y,Z and Normal Components at Each Point.xlsx"  
output_df.to_excel(excel3_path, index=False)

print(f"Output saved successfully at: {excel3_path}")